In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
pd.set_option('max_columns',30)

In [2]:
df=pd.read_csv('data/hubNashville__311__Service_Requests.csv')
original_df=df[['Request #','City','ZIP','Latitude','Longitude']]
original_df.columns = ['request_id','city','zip','lat','long']
original_df.city = original_df.city.str.lower()

/Users/sbussa/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/sbussa/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [3]:
original_df.request_id.is_unique
original_df = original_df.set_index('request_id')

In [4]:
trash_report=pd.read_excel(open('data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx','rb'),
                                               sheet_name='Trash Hauler Report',header=1)


trash_report.columns = trash_report.columns.str.lower()
trash_report.columns= trash_report.columns.str.replace(' ','_')
trash_report.rename(columns={'request_':'request'},inplace = True)
trash_report.rename(columns={'request_number':'request_id'},inplace = True)
columns_to_drop=['state_plan_x','state_plan_y']
trash_report.drop(columns=columns_to_drop,axis=1,inplace=True)
trash_report.incident_address = trash_report.incident_address.str.lower()

trash_report.set_index('request_id',inplace=True)
trash_report = pd.merge(trash_report,original_df,on='request_id',how='left')
trash_report['incident_address']=trash_report['incident_address'].apply(lambda x: str(x).split(',',1)[0])
trash_report.city=trash_report.city.replace('nashville, tn','nashville')
trash_report.description=trash_report.description.str.lower()
trash_report.request=trash_report.request.str.lower()
trash_report=trash_report.fillna('NA')
trash_report=trash_report[trash_report.trash_route != 'NA']#delete rows where trash_route is NA
trash_report.city = trash_report.city.str.lower()

In [5]:
trash_report['missed_pickups']=trash_report.request+trash_report.description
trash_report['pickup_day']=trash_report.trash_route.apply(lambda x: str(x)[1])
trash_report['request_date']=trash_report.date_opened.apply(lambda x: x.weekday())
trash_report.request_date=[x+1 for x in trash_report.request_date]

when we get the weekday from datettime, we get the 0 for monday and 1 for tuesday. To synch with the users pickup days structure( 2-Tuesday, 3-wed). we need to make both common.

In [6]:
#trash_report.missed_pickups=trash_report.missed_pickups.str.lower()

In [7]:
def pick_day(day):
    x=None
    if day[1]=='2':
        x= 'Tuesday'
    elif day[1]=='3':
        x= 'Wednesday'
    elif day[1]=='4':
        x= 'Thursday'
    elif day[1]=='5':
        x= 'Friday'
    return x

In [8]:
trash_report_backup=trash_report

In [9]:
def get_seventh_day(pickupday,reqday,request_date):
    request_date=pd.to_datetime(request_date)
    diff = abs(reqday-pickupday)

    if diff==0:
        seventh_day=request_date+timedelta(days=6)
    elif diff>0:
        request_date=request_date-timedelta(days=diff)
        seventh_day=request_date+timedelta(days=6)

    #print('diff:'+ str(diff)+' pickup_date:' + str(request_date) + '' + ' Seventh_day:'+str(seventh_day))
    return seventh_day


In [10]:
trash_report['seventh_day']=trash_report.apply(lambda row: get_seventh_day(int(row['pickup_day']),
                                                                           int(row['request_date']),
                                                                               row['date_opened']
                                                                           ),axis=1)
 

In [11]:
explore_trash=trash_report[['date_opened','seventh_day','request','description','incident_address','zip_code','trash_route','trash_hauler']]
explore_trash['date_opened']=[d.date() for d in explore_trash["date_opened"]]
explore_trash['seventh_day']=[d.date() for d in explore_trash['seventh_day']] 
explore_trash.reset_index(inplace = True)

/Users/sbussa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sbussa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
explore_trash_counts=explore_trash.groupby(['incident_address','request','seventh_day'])['request_id'].count()
explore_trash_counts=pd.DataFrame(explore_trash_counts)
explore_trash_counts.reset_index(inplace=True)
explore_trash_counts.columns = ['incident_address','request','seventh_day','counts']

In [13]:
explore_trash_counts

,incident_address,request,seventh_day,counts
0,1 belle forrest ave c,trash - curbside/alley missed pickup,2018-07-05,1
1,10 belle forrest ave,trash - curbside/alley missed pickup,2018-07-04,1
2,100 bluefield square,trash - curbside/alley missed pickup,2019-03-21,1
3,100 braxton hill ct,trash - curbside/alley missed pickup,2018-03-13,1
4,100 brook hollow rd,trash - curbside/alley missed pickup,2018-12-11,1
5,100 cedarmont ct,trash - curbside/alley missed pickup,2017-12-02,1
6,100 clydelan ct,trash - curbside/alley missed pickup,2019-02-19,2
7,100 desoto dr,trash - curbside/alley missed pickup,2019-11-05,1
8,100 dyer ct,trash - curbside/alley missed pickup,2019-11-04,1
9,100 graeme dr,trash - curbside/alley missed pickup,2018-12-13,1


In [14]:
duplicates=explore_trash_counts[explore_trash_counts.counts>1].sort_values(by='counts',ascending=False)#483 duplicate rows needs to be deleted

In [15]:
duplicates.shape

(483, 4)

In [16]:
all_dups=pd.merge(explore_trash,duplicates,on = ['incident_address','request','seventh_day'],how ='outer')
all_dups=all_dups.sort_values('date_opened')
all_dups_cleaned=all_dups.drop_duplicates(subset = ['incident_address','request','seventh_day'],keep = 'first') 

In [17]:
 #duplicates.groupby('incident_address').size().loc[lambda x: x>1].sort_values()

This address is the most which called the duplicates 4 times: "802 crescent rd"

In [18]:
#explore_trash[explore_trash.incident_address=='802 crescent rd'].sort_values(by='date_opened')

In [19]:
all_dups_cleaned.shape#18770 rows

(18770, 10)

In [20]:
trash_report[trash_report.incident_address=='4311 dakota ave']

,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,city,zip,lat,long,missed_pickups,pickup_day,request_date,seventh_day
request_id,,,,,,,,,,,,,,,,
25512,2017-11-01 16:52:00,trash - curbside/alley missed pickup,cust says entire street was missed,4311 dakota ave,37209,RED RIVER,1308,24,nashville,37209,36.1478,-86.8413,trash - curbside/alley missed pickupcust says ...,3,3,2017-11-07 16:52:00
25586,2017-11-02 08:34:00,trash - curbside/alley missed pickup,missed pickup,4311 dakota ave,37209,RED RIVER,1308,24,nashville,37209,36.1475,-86.8392,trash - curbside/alley missed pickupmissed pickup,3,4,2017-11-07 08:34:00


In [21]:
all_dups_cleaned.head()

,request_id,date_opened,seventh_day,request,description,incident_address,zip_code,trash_route,trash_hauler,counts
0,25270,2017-11-01,2017-11-06,trash - backdoor,"house with the wheel chair ramp, they share dr...",3817 crouch dr,37207,3205,RED RIVER,NaN
20,25469,2017-11-01,2017-11-07,trash - curbside/alley missed pickup,"our trash was not picked up today, wed, nov 1....",117 lafayette ct,37205,1302,RED RIVER,NaN
21,25471,2017-11-01,2017-11-06,trash - curbside/alley missed pickup,"alley trash miss tuesday, also others as well,...",1904 meharry blvd,37208,9208,METRO,NaN
22,25474,2017-11-01,2017-11-07,trash - curbside/alley missed pickup,customer says backdoor pickup was missed the p...,1144 sewanee rd,37220,3302,RED RIVER,NaN
23,25496,2017-11-01,2017-11-07,trash - curbside/alley missed pickup,missed pickup,1815 woodmont blvd,37215,3305,RED RIVER,NaN


In [22]:
official_cln_trash_report=pd.merge(trash_report,all_dups_cleaned,on='request_id',how='inner')

In [23]:
official_cln_trash_report.head()

,request_id,date_opened_x,request_x,description_x,incident_address_x,zip_code_x,trash_hauler_x,trash_route_x,council_district,city,zip,lat,long,missed_pickups,pickup_day,request_date,seventh_day_x,date_opened_y,seventh_day_y,request_y,description_y,incident_address_y,zip_code_y,trash_route_y,trash_hauler_y,counts
0,25270,2017-11-01 07:09:00,trash - backdoor,"house with the wheel chair ramp, they share dr...",3817 crouch dr,37207,RED RIVER,3205,2,nashville,37207,36.2173,-86.8154,trash - backdoorhouse with the wheel chair ram...,2,3,2017-11-06 07:09:00,2017-11-01,2017-11-06,trash - backdoor,"house with the wheel chair ramp, they share dr...",3817 crouch dr,37207,3205,RED RIVER,NaN
1,25274,2017-11-01 07:42:00,trash - curbside/alley missed pickup,curb/trash miss tuesday.,4028 clarksville pike,37218,RED RIVER,4202,1,nashville,37218,36.2135,-86.8381,trash - curbside/alley missed pickupcurb/trash...,2,3,2017-11-06 07:42:00,2017-11-01,2017-11-06,trash - curbside/alley missed pickup,curb/trash miss tuesday.,4028 clarksville pike,37218,4202,RED RIVER,NaN
2,25276,2017-11-01 07:53:00,trash - curbside/alley missed pickup,curb/trash miss tuesday.,6528 thunderbird dr,37209,RED RIVER,4205,20,nashville,37209,36.1429,-86.8856,trash - curbside/alley missed pickupcurb/trash...,2,3,2017-11-06 07:53:00,2017-11-01,2017-11-06,trash - curbside/alley missed pickup,curb/trash miss tuesday.,6528 thunderbird dr,37209,4205,RED RIVER,NaN
3,25307,2017-11-01 08:28:00,trash - curbside/alley missed pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,nashville,37207,36.2127,-86.7892,trash - curbside/alley missed pickupmissed,2,3,2017-11-06 08:28:00,2017-11-01,2017-11-06,trash - curbside/alley missed pickup,missed,2603 old matthews rd,37207,2206,WASTE IND,NaN
4,25312,2017-11-01 08:55:00,trash - curbside/alley missed pickup,missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,nashville,37209,36.1549,-86.875,trash - curbside/alley missed pickupmissed the...,2,3,2017-11-06 08:55:00,2017-11-01,2017-11-06,trash - curbside/alley missed pickup,missed the even side of the road.,604 croley dr,37209,4203,RED RIVER,NaN


In [24]:
official_cln_trash_report=official_cln_trash_report[['request_id',
                                                     'date_opened_y',
                                                     'seventh_day_y',
                                                     'request_x',
                                                     'description_y',
                                                     'missed_pickups',
                                                     'trash_hauler_y','trash_route_y','pickup_day',
                                                     'council_district','incident_address_y','city','zip','lat','long']]

In [25]:
official_cln_trash_report.columns=['request_id',
                                   'date_opened',
                                   'seventh_day',
                                   'request',
                                   'description',
                                   'missed_pickups',
                                   'trash_hauler','trash_route','pickup_day',
                                   'council_district','incident_address','city','zip','lat','long']

In [26]:
official_cln_trash_report=official_cln_trash_report.sort_values(by='date_opened',ascending=True)

In [27]:
official_cln_trash_report[official_cln_trash_report.request=='trash collection complaint']

,request_id,date_opened,seventh_day,request,description,missed_pickups,trash_hauler,trash_route,pickup_day,council_district,incident_address,city,zip,lat,long
32,25519,2017-11-01,2017-11-07,trash collection complaint,"5th time in as many weeks, metro has not picke...",trash collection complaint5th time in as many ...,RED RIVER,3304,3,34,3702 hobbs rd,nashville,37215,36.103,-86.8367
34,25525,2017-11-01,2017-11-07,trash collection complaint,missed my trash again.\ncheck your records. i...,trash collection complaintmissed my trash agai...,RED RIVER,4304,3,16,162 antioch pike,nashville,37211,36.1011,-86.729
26,25508,2017-11-01,2017-11-07,trash collection complaint,missed. neighbor missed as well-say she is at ...,trash collection complaintmissed. neighbor mis...,RED RIVER,2305,3,18,2812 27th ave s,nashville,37212,36.1245,-86.8109
16,25393,2017-11-01,2017-11-05,trash collection complaint,trash carts all tossed in road after pick up,trash collection complainttrash carts all toss...,RED RIVER,1501,5,15,2720 riverbend dr,nashville,37214,36.2357,-86.6987
17,25407,2017-11-01,2017-11-07,trash collection complaint,curb/trash miss everyone already gotten pick u...,trash collection complaintcurb/trash miss ever...,RED RIVER,3305,3,25,4209 lone oak rd,nashville,37215,36.1009,-86.8095
7,25327,2017-11-01,2017-11-06,trash collection complaint,"trash out on time, miss again tuesday. alley","trash collection complainttrash out on time, m...",METRO,9208,2,21,1816 jo johnston ave,nashville,37203,36.1603,-86.803
5,25317,2017-11-01,2017-11-07,trash collection complaint,left trash cart in middle of driveway instead ...,trash collection complaintleft trash cart in m...,RED RIVER,4304,3,16,3602 floral dr,nashville,37211,36.0981,-86.734
11,25350,2017-11-01,2017-11-06,trash collection complaint,missed- 4th week in a row.,trash collection complaintmissed- 4th week in ...,RED RIVER,3201,2,3,3210 whites creek pike,nashville,37207,36.2314,-86.8208
13,25366,2017-11-01,2017-11-06,trash collection complaint,trash truck took cart \n3 guys on truck did no...,trash collection complainttrash truck took car...,METRO,9201,2,21,2200 osage st,nashville,37208,36.1769,-86.8127
63,25784,2017-11-02,2017-11-06,trash collection complaint,customer says it was missed again,trash collection complaintcustomer says it was...,METRO,9203,2,5,608 cleveland st,nashville,37207,36.1858,-86.761


In [28]:
official_cln_trash_report.date_opened = pd.to_datetime(official_cln_trash_report.date_opened)

In [32]:
official_cln_trash_report.council_district


0         2
20       23
21       21
22       34
23       25
24       34
25       26
27       24
28       16
29       24
30       25
31       26
32       34
33       23
34       16
35       34
19       20
18        1
26       18
16       15
17       25
1         1
2        20
3         2
4        20
6        34
7        21
8        33
5        16
10        2
         ..
18764    15
18765    32
18766     6
18747    13
18757     5
18746    29
18735    27
18744     8
18768    32
18724    25
18725     8
18726    18
18727    25
18728    32
18729     6
18730    15
18731    17
18745    31
18732    17
18734     7
18736    18
18737     8
18738    18
18739    27
18740    19
18741    17
18742    21
18743    17
18733    19
18769    29
Name: council_district, Length: 18770, dtype: object

In [ ]:
#official_cln_trash_report.groupby(['incident_address','trash'])
df_missed_pickups=official_cln_trash_report[official_cln_trash_report.missed_pickups.str.contains('missed|miss')]
df_missed_curbside=official_cln_trash_report[official_cln_trash_report.missed_pickups.str.contains('trash - curbside/alley')]

In [ ]:
df_missed_pickups.trash_hauler.value_counts()

In [ ]:
df_missed_pickups.groupby(['incident_address','request','council_district','trash_hauler','trash_route']).size()

In [ ]:
df_missed_pickups.head()

In [ ]:
six_months_duplicates=official_cln_trash_report.groupby('incident_address').rolling(on='date_opened',window='180D').request_id.count()
six_months_duplicates = pd.DataFrame(six_months_duplicates)
six_months_duplicates.reset_index(inplace=True)
six_months_duplicates.sort_values(['date_opened','incident_address'],ascending=False)
six_months_duplicates.columns= ['incident_address','date_opened','counts']

In [ ]:
six=official_cln_trash_report.groupby('incident_address').rolling(on='date_opened',window='180D').request_id.count()

In [ ]:
six_months_duplicates.head() 

In [ ]:
six_months_duplicates['is_charged'] =  six_months_duplicates.counts.apply(lambda x:  True if( x>2 ) else False ) 
 

In [ ]:
six_months_duplicates.shape

In [ ]:
six_months_duplicates=pd.merge(six_months_duplicates,official_cln_trash_report,on = ['incident_address','date_opened'],how ='outer' )

In [ ]:
six_months_dups_time_series = six_months_duplicates.set_index('date_opened')

In [ ]:
metro=six_months_dups_time_series[six_months_dups_time_series.trash_hauler=='metro']

In [ ]:
metro.head()

In [ ]:
six_months_dups_time_series.counts.plot()

In [ ]:
six_months_duplicates=six_months_duplicates[six_months_duplicates.request_id>2]

In [ ]:
six_months_duplicates.shape#2297 in 6 months rolling window. 

In [ ]:
six_months_duplicates

In [ ]:
six_months_duplicates.head()

In [ ]:
trash_report[trash_report.incident_address=='4807 nolensville pike'].sort_values(by='date_opened')

In [ ]:
official_cln_trash_report.shape

In [ ]:
official_cln_trash_report.trash_hauler = official_cln_trash_report.trash_hauler.str.lower()

In [ ]:
official_cln_trash_report.trash_hauler.value_counts()

In [ ]:
official_cln_trash_report.zip.value_counts()

In [ ]:
def is_zipGreater(x):
    if x=='NA':
        continue
    x=int(x)
    elif x>37208:
        return 'red'
    else:
        return 'blue'

In [ ]:
official_cln_trash_report['is_red_or_blue']=official_cln_trash_report.zip.apply(is_zipGreater)